<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Analisis_de_Inclinometros_(GKN)_datos_crudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL - Procesamiento de datos de inclinómetros
## Paso 1: Carga y concatenación de datos crudos

Este notebook procesa las lecturas de inclinómetros almacenadas en archivos `.gkn`,
extrayendo la información relevante y consolidándola en un solo `DataFrame`.

### Objetivos:
1. Leer múltiples archivos `.gkn` y extraer datos clave.
2. Unificar las lecturas en un `DataFrame`, incorporando fecha, hora e identificadores del proyecto.
3. Guardar los datos procesados en un archivo `.csv`.

---

In [17]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        df.insert(0, 'inclinometro', hole_no)
        df['project'] = project
        df['probe_no'] = probe_no
        df.insert(1, 'fecha_hora', fecha_hora)
        df.insert(2, 'Punto de Lectura', range(1, len(df) + 1))
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

if not dfs:
    print("No se encontraron archivos válidos o ningún archivo tiene datos procesables.")
else:
    df_total = pd.concat(dfs, ignore_index=True)
    # Extraer solo la fecha (sin hora)
    df_total['solo_fecha'] = pd.to_datetime(df_total['fecha_hora']).dt.date

    # Obtener las fechas únicas en orden ascendente
    fechas_ordenadas = sorted(df_total['solo_fecha'].unique())

    # Crear un diccionario de fecha a número de lectura correlativo
    mapa_lectura_nro = {fecha: nro+1 for nro, fecha in enumerate(fechas_ordenadas)}

    # Asignar el número de lectura según la fecha
    df_total['Lectura Nro'] = df_total['solo_fecha'].map(mapa_lectura_nro)

    # Reorganizar columnas: inclinometro, fecha_hora, Lectura Nro, Punto de Lectura, ...
    columnas_finales = ['inclinometro', 'fecha_hora', 'Lectura Nro', 'Punto de Lectura']
    resto = [c for c in df_total.columns if c not in columnas_finales + ['solo_fecha']]
    columnas_finales += resto
    df_total = df_total[columnas_finales]

    # Eliminar la columna auxiliar 'solo_fecha' (sin error si no existe)
    df_total = df_total.drop(columns=['solo_fecha'], errors='ignore')

    # Renombrar columnas según lo solicitado
    df_total = df_total.rename(columns={
        'inclinometro': 'Inclinometro',
        'fecha_hora': 'Fecha-Hora',        # <--- Cambio aquí
        'Lectura Nro': 'Lectura',
        'Punto de Lectura': 'Puntos Leidos',
        'project': 'Proyecto',
        'probe_no': 'Sonda'
    })

    output_csv = '/content/lecturas_inclinometro_unificado.csv'
    df_total.to_csv(output_csv, index=False)
    print(f"Archivo CSV guardado en: {output_csv}")
    display(df_total.head())

<ipython-input-17-ce06038fdb35>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-17-ce06038fdb35>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-17-ce06038fdb35>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-17-ce06038fdb35>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_wh

Archivo CSV guardado en: /content/lecturas_inclinometro_unificado.csv


,Inclinometro,Fecha-Hora,Lectura,Puntos Leidos,FLEVEL,A+,A-,B+,B-,Proyecto,Sonda
0,MD-IN1-A1,2019-03-25 17:10:08,1,1,91.5,-130,123.0,-165,183.0,MD,p 12
1,MD-IN1-A1,2019-03-25 17:10:08,1,2,91.0,-103,96.0,-116,156.0,MD,p 12
2,MD-IN1-A1,2019-03-25 17:10:08,1,3,90.5,-77,71.0,-75,111.0,MD,p 12
3,MD-IN1-A1,2019-03-25 17:10:08,1,4,90.0,-35,34.0,-44,87.0,MD,p 12
4,MD-IN1-A1,2019-03-25 17:10:08,1,5,89.5,-34,30.0,-73,118.0,MD,p 12


In [21]:
import pandas as pd

# URL del CSV consolidado de lecturas
csv_url = "https://raw.githubusercontent.com/JorgeAccardi/auscultacion-presa/main/lecturas_inclinometro_unificado.csv"

# Cargar el archivo CSV
df = pd.read_csv(csv_url)

# Mostrar el nombre de las columnas y las primeras filas para inspección
print("Columnas del DataFrame:")
print(df.columns.tolist())

print("\nPrimeras filas del archivo:")
display(df.head())

# Mostrar agrupación por Inclinómetro y Fecha-Hora (primeros valores)
print("\nCantidad de campañas por Inclinometro:")
print(df.groupby("Inclinometro")["Fecha-Hora"].nunique())

print("\nValores únicos de profundidad (Puntos Leidos):")
print(sorted(df["Puntos Leidos"].unique()))

Columnas del DataFrame:
['Inclinometro', 'Fecha-Hora', 'Lectura', 'Puntos Leidos', 'FLEVEL', 'A+', 'A-', 'B+', 'B-', 'Proyecto', 'Sonda']

Primeras filas del archivo:


,Inclinometro,Fecha-Hora,Lectura,Puntos Leidos,FLEVEL,A+,A-,B+,B-,Proyecto,Sonda
0,MD-IN1-A1,2019-03-25 17:10:08,1,1,91.5,-130,123.0,-165,183.0,MD,p 12
1,MD-IN1-A1,2019-03-25 17:10:08,1,2,91.0,-103,96.0,-116,156.0,MD,p 12
2,MD-IN1-A1,2019-03-25 17:10:08,1,3,90.5,-77,71.0,-75,111.0,MD,p 12
3,MD-IN1-A1,2019-03-25 17:10:08,1,4,90.0,-35,34.0,-44,87.0,MD,p 12
4,MD-IN1-A1,2019-03-25 17:10:08,1,5,89.5,-34,30.0,-73,118.0,MD,p 12



Cantidad de campañas por Inclinometro:
Inclinometro
MD-IN1-A1    93
Name: Fecha-Hora, dtype: int64

Valores únicos de profundidad (Puntos Leidos):
[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int